In [10]:
from itertools import combinations
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import clear_output, display
import sys
sys.path.append('../scripts') 
from functions import *
from config import nutrient_info, conversion_factors

In [11]:
df_food_scaled = pd.read_csv('../data/df_food_scaled.csv')

C:\Users\Sophie\AppData\Local\Temp\ipykernel_15168\2711278687.py:1: DtypeWarning: Columns (1,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food_scaled = pd.read_csv('../data/df_food_scaled.csv')


In [12]:
qualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'qualifying'}
disqualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'disqualifying'}

In [13]:
df_bread_pastries = df_food_scaled[df_food_scaled['food_group'] == 'bread_pastries']
df_bread_pastries.drop_duplicates('combined_name')[['combined_name', 'QI', 'DI', 'NB']].sort_values('QI', ascending=False)

,combined_name,QI,DI,NB
283251,Migros Baguette,36.236199,0.834880,49.914477
294750,Schär Pain Campagnard gluten-free WHEAT-FREE L...,36.104429,0.849150,51.251474
285797,Schär Meisterbäckers,34.620562,0.758124,50.362291
271276,Gluten-free bread,31.682763,0.527422,47.912963
284903,Schär Meisterbäckers Vital,31.612550,0.784880,49.404500
...,...,...,...,...
276341,Bun raisins,0.036254,0.142020,3.625378
190890,Almond croissant,0.035276,0.216709,3.527648
309555,Coop Qualité & Prix - Weiss,0.022450,1.146663,2.245005
188766,Churros,0.019833,0.438044,1.983270


In [14]:
high_qi_bread_pastries = df_bread_pastries[df_bread_pastries['QI'] > 2].drop_duplicates('combined_name').sort_values('QI', ascending=False)

In [15]:
high_qi_bread_pastries[['combined_name', 'eaten_quantity', 'QI', 'DI', 'NB']]

,combined_name,eaten_quantity,QI,DI,NB
283251,Migros Baguette,110.0,36.236199,0.834880,49.914477
294750,Schär Pain Campagnard gluten-free WHEAT-FREE L...,110.0,36.104429,0.849150,51.251474
285797,Schär Meisterbäckers,110.0,34.620562,0.758124,50.362291
271276,Gluten-free bread,50.0,31.682763,0.527422,47.912963
284903,Schär Meisterbäckers Vital,50.0,31.612550,0.784880,49.404500
313779,M Classic - Bun Di,250.0,4.798419,0.589957,44.528986


In [16]:
ratio_cols = [i for i in high_qi_bread_pastries.columns if '_ratio_scaled' in i]

df_corr = high_qi_bread_pastries[ratio_cols + ['QI']]
corr = df_corr.corr()['QI']
corr.sort_values(ascending=True)

niacin_eaten_ratio_scaled                                   -0.985530
folate_eaten_ratio_scaled                                   -0.984752
sugar_eaten_ratio_scaled                                    -0.984631
pantothenic_acid_eaten_ratio_scaled                         -0.984508
fiber_eaten_ratio_scaled                                    -0.972355
vitamin_c_eaten_ratio_scaled                                -0.971705
protein_eaten_ratio_scaled                                  -0.711251
fat_eaten_ratio_scaled                                      -0.270950
salt_eaten_ratio_scaled                                      0.657810
fatty_acids_saturated_eaten_ratio_scaled                     0.670151
calcium_eaten_ratio_scaled                                   0.999754
sodium_eaten_ratio_scaled                                    0.999754
zinc_eaten_ratio_scaled                                      0.999754
iron_eaten_ratio_scaled                                      0.999754
vitamin_e_activity_e

In [17]:
def interactive_exclude_nutrient(nutrient):
    clear_output(wait=True)
    df_plot = compare_qi_excluding_nutrient(
        df=high_qi_bread_pastries, 
        nutrient_to_exclude=nutrient,
        qualifying_nutrients=qualifying_nutrients, 
        scaling_factor=2000
    )

    display(df_plot[['combined_name', 'QI', f"QI_excl_{nutrient}", 'QI_diff', 'QI_pct_change']])

    print(f"\nExcluding {nutrient} for NB:")
    df_nb = compare_nb_excluding_nutrient(
        df=high_qi_bread_pastries, 
        nutrient_to_exclude=nutrient,
        qualifying_nutrients=qualifying_nutrients, 
        scaling_factor=2000
    )
    display(df_nb[['combined_name', 'NB', f"NB_excl_{nutrient}", 'NB_diff', 'NB_pct_change']])

nutrient_choice = ['folate_eaten', 
                   'niacin_eaten', 
                   'pantothenic_acid_eaten', 
                   'vitamin_b2_eaten', 
                   'vitamin_b1_eaten', 
                   'all_trans_retinol_equivalents_activity_eaten', 
                   'beta_carotene_eaten', 
                   'vitamin_b12_eaten', 
                   'vitamin_b6_eaten', 
                   'vitamin_c_eaten', 
                   'vitamin_d_eaten', 
                   'vitamin_e_activity_eaten', 
                   'calcium_eaten', 
                   'iron_eaten', 
                   'magnesium_eaten', 
                   'phosphorus_eaten', 
                   'potassium_eaten', 
                   'zinc_eaten', 
                   'fiber_eaten', 
                   'protein_eaten']

In [18]:
interact(interactive_exclude_nutrient, nutrient=widgets.Dropdown(options=nutrient_choice, description='Exclude:'))

interactive(children=(Dropdown(description='Exclude:', options=('folate_eaten', 'niacin_eaten', 'pantothenic_a…

<function __main__.interactive_exclude_nutrient(nutrient)>

- vitamin b1
- vitamin b12
- vitamin b6

In [19]:
exclude_list = ['vitamin_b1_eaten', 'vitamin_b6_eaten', 'vitamin_b12_eaten']
qualifying_nutrients = list(nutrient_info.keys())

for idx, row in high_qi_bread_pastries.iterrows():

    qi_incl = compute_qi_excluding_multiple(row, qualifying_nutrients, exclude_list=[])
    qi_excl = compute_qi_excluding_multiple(row, qualifying_nutrients, exclude_list=exclude_list)

    nb_incl = compute_nb_excluding_multiple(row, qualifying_nutrients, exclude_list=[]) 
    nb_excl = compute_nb_excluding_multiple(row, qualifying_nutrients, exclude_list=exclude_list)

    print(f"Food item : {row['combined_name']}")
    print(f"QI (incl. all): {qi_incl}")
    print(f"QI (excl. {', '.join(exclude_list)}): {qi_excl}")
    print(f"NB (incl. all): {nb_incl:.2f}")
    print(f"NB (excl. {', '.join(exclude_list)}): {nb_excl:.2f}")
    print("-----------")

Food item : Migros Baguette
QI (incl. all): 29.18552087057083
QI (excl. vitamin_b1_eaten, vitamin_b6_eaten, vitamin_b12_eaten): 0.7434736511261102
NB (incl. all): 49.69
NB (excl. vitamin_b1_eaten, vitamin_b6_eaten, vitamin_b12_eaten): 42.83
-----------
Food item : Schär Pain Campagnard gluten-free WHEAT-FREE LACTOSE-FREE
QI (incl. all): 29.08354615513947
QI (excl. vitamin_b1_eaten, vitamin_b6_eaten, vitamin_b12_eaten): 0.7691736026137956
NB (incl. all): 52.35
NB (excl. vitamin_b1_eaten, vitamin_b6_eaten, vitamin_b12_eaten): 45.85
-----------
Food item : Schär Meisterbäckers
QI (incl. all): 27.87476476214078
QI (excl. vitamin_b1_eaten, vitamin_b6_eaten, vitamin_b12_eaten): 0.7461989968782841
NB (incl. all): 51.84
NB (excl. vitamin_b1_eaten, vitamin_b6_eaten, vitamin_b12_eaten): 45.27
-----------
Food item : Gluten-free bread
QI (incl. all): 25.46946391684589
QI (excl. vitamin_b1_eaten, vitamin_b6_eaten, vitamin_b12_eaten): 0.6200012067597128
NB (incl. all): 46.69
NB (excl. vitamin_b1_ea